# Listeners must have arguments "event" and "metadata" with proper types

In [1]:
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.infrastructure.event_sourcing.EventPublisher import EventPublisher, listen_to


class Invalid:
  pass


class ProjectorInvalid0:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self):
    print('Created project')


class ProjectorInvalid1:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, invalid: ProjectCreatedEvent, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid2:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, invalid: EventMetadata):
    print('Created project')


class ProjectorInvalid3:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: Invalid, metadata: EventMetadata):
    print('Created project')


class ProjectorInvalid4:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: Invalid):
    print('Created project')


event_publisher = EventPublisher()
try:
  projector = ProjectorInvalid0()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid1()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid2()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid3()
  event_publisher.register(projector)
except Exception as e:
  print(e)

try:
  projector = ProjectorInvalid4()
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event listener <bound method ProjectorInvalid0.on_project_created of <__main__.ProjectorInvalid0 object at 0x13b1f7b00>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid1.on_project_created of <__main__.ProjectorInvalid1 object at 0x13b4d3140>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid2.on_project_created of <__main__.ProjectorInvalid2 object at 0x13b1f7b00>> must have an argument called "metadata" that should by type hinted with a subclass of EventMetadata
Event listener <bound method ProjectorInvalid3.on_project_created of <__main__.ProjectorInvalid3 object at 0x13b4d3140>> must have an argument called "event" that should by type hinted with a subclass of EventBase
Event listener <bound method ProjectorInvalid4.on_project_created of <__main__.ProjectorInvalid4 object at 0x13b1f7b00>> must have an arg

# Cannot register same listener twice

In [2]:
class Projector:
  @listen_to(ProjectCreatedEvent)
  def on_project_created(self, event: ProjectCreatedEvent, metadata: EventMetadata):
    print(f'Created project {event.get_project_id()} with metadata {metadata}')


try:
  event_publisher = EventPublisher()
  projector = Projector()
  event_publisher.register(projector)
  event_publisher.register(projector)
except Exception as e:
  print(e)

Event handler already registered


# Create projects

In [3]:
from morpheus.common.types.identity.Identity import UserId
from morpheus.project.infrastructure.persistence.ProjectSummaryRepository import project_summary_repository
from morpheus.project.infrastructure.event_sourcing.ProjectEventStore import project_event_store
from morpheus.project.types.Project import ProjectId, Name, Description, Tags
from morpheus.project.application.write.Project import CreateProjectCommand, CreateProjectCommandHandler

for i in range(10):
  create_project_command = CreateProjectCommand(
    project_id=ProjectId.new(),
    name=Name.from_str(f'Project {i}'),
    description=Description.from_str(f'Description {i}'),
    tags=Tags.from_list([f'tag{i}', f'tag{i + 1}']),
    user_id=UserId.new()
  )
  CreateProjectCommandHandler.handle(create_project_command)

event_envelopes = project_event_store.get_all_events_ordered_by_version()
print(f'Found {len(event_envelopes)} events')
for event_envelope in event_envelopes:
  print(f'"{event_envelope.get_event().get_event_name().to_str()}" occurred at {event_envelope.event.get_occurred_at().to_iso_with_timezone()}')

project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 584 events
"Project Created" occurred at 2026-02-16T19:04:30.238135-03:00
"Project Created" occurred at 2026-02-16T19:04:30.467500-03:00
"Project Created" occurred at 2026-02-16T19:04:30.684756-03:00
"Project Created" occurred at 2026-02-16T19:04:30.904244-03:00
"Project Created" occurred at 2026-02-16T19:04:31.131681-03:00
"Project Created" occurred at 2026-02-16T19:04:31.360692-03:00
"Project Created" occurred at 2026-02-16T19:04:31.615617-03:00
"Project Created" occurred at 2026-02-16T19:04:31.846971-03:00
"Project Created" occurred at 2026-02-16T19:04:32.077973-03:00
"Project Created" occurred at 2026-02-16T19:04:32.307777-03:00
"Project Created" occurred at 2026-02-16T19:04:32.569202-03:00
"Project Created" occurred at 2026-02-16T19:04:32.805824-03:00
"Project Created" occurred at 2026-02-16T19:04:33.309665-03:00
"Project Created" occurred at 2026-02-16T19:04:33.692003-03:00
"Project Created" occurred at 2026-02-16T19:04:33.919744-03:00
"Project Created" occurred at 2026-02-

# Reproject project summaries

In [4]:
from morpheus.project.presentation.cli.ProjectionCliCommands import ReprojectProjectSummariesCliCommand

ReprojectProjectSummariesCliCommand.run()

Reprojecting project summaries


------------------------------


Found 584 events


Reset projection


Successfully reprojected project summaries


In [5]:
project_summaries = project_summary_repository.find_all()
print(f'Found {len(project_summaries)} project summaries')
for project_summary in project_summaries:
  print(f'Project {project_summary.project_id.to_str()}: {project_summary.project_name.to_str()}')

Found 96 project summaries
Project b45f506e-1e2d-42b5-9078-9ba3e2f684d5: Test Project for Layers
Project aec64885-53ac-42ba-beb0-6e2db6d56460: Test Project for Layers
Project 05d79e4d-a29d-4122-aa97-a4ffca503991: Test Project for Layers
Project c5140249-55a3-409a-afb2-cd8ad00e8f9d: Test Project for Layers
Project 94d4d5e6-eb45-4b46-9c5d-4c50309c8368: Test Project for Layers
Project 4221451d-d3ef-4368-bbe1-477a9c3ec24f: Test Project for Layers
Project 8bc91204-e82f-49f1-88ec-f8422d749784: Test Project for Layers
Project 70b5e492-bb7b-4bd0-bb54-e3aa8b28d3bf: Test Project for Layers
Project ad24ff58-07f7-4e30-b986-60179302018b: Test Project for Layers
Project 84e6de14-9c50-4819-9fe1-e9573a260bde: Test Project for Layers
Project af616b15-032e-4a1a-9d94-b6ce95ed4edc: Test Project for Layers
Project 94a5d3f6-9f92-4bba-9348-1dc5f24dc43e: Test Project for Layers
Project 3609137e-d029-40c1-955b-0f7798256938: Test Project for Layers
Project 61ec7a4c-55b3-44d8-ad38-c4bac66ba431: Test Project
Proj